<a href="https://colab.research.google.com/github/joshuazhu17/LinearKernelMDP/blob/main/UCLKRecreation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy
#import matplotlib.pylab as plt
#import torch
import math
import copy
import random
import cvxpy as cp
from random import randrange
#from matplotlib.backends.backend_pdf import PdfPages

In [ ]:
dim = 8
ACTION = np.power(2,dim-1)
STATE = 2
gamma = 0.5
T = 10000
thetastar = np.append(0.05*np.ones(dim-1), 1)
delta = 1-gamma

In [ ]:
from abc import ABC, abstractmethod

In [ ]:
class Env(ABC):
  '''
  make this function do a step in the environment
  @abstractmethod
  def transition(self, state, action):
    pass
  '''

  @abstractmethod
  def transition_prob(self, state, action, next_state):
    pass

  @abstractmethod
  def reward(self, state, action):
    pass

  @abstractmethod
  def step(self, action):
    pass

  @abstractmethod
  def reset(self):
    pass

In [ ]:
# Environment is not necessarily Linear Kernel MDP
# Environment should only have transition function and rewards function
# Everything else should be in the algorithm class, if at all

class LinearKernelMDPEnv(Env):
  def __init__(self, d, A, S, gamma, theta):
    self.dim = d
    self.num_actions = A
    self.num_states = S
    self.gamma = gamma
    self.theta = theta

  # this is the phi function
  @abstractmethod
  def embed(self, state, action, next_state):
    pass

  # this is the phiv function
  @abstractmethod
  def weighted_value_embedding(self, state, action, values):
    pass

  # this is a test to make sure it is a Linear Kernel MDP
  def is_valid(self):
    # 1) makes sure norm(theta) <= sqrt(d)
    # 2) make sure dot(phi(s, a, s'), theta) makes a valid probability distribution
    # 3) make sure weighted_value_embedding <= sqrt(d)R, where R is the max reward of values
    pass


In [ ]:
class DongRuoToy(LinearKernelMDPEnv):
  def __init__(self, d, A, S, gamma, theta):
    super().__init__(d, A, S, gamma, theta)
    self.delta = 1 - gamma
    self.cur_state = 0

  def trans_action(self, a, d):###d is dimension, dimension of a is d-1
    tt = np.zeros(d-1)-1
    bb = bin(a)
    for i in range(len(bb)-2):
        tt[i] = 2*np.double(bb[i+2])-1
    return(tt)

  def embed(self, state, action, next_state):
    tt = np.zeros(self.dim)
    aa = self.trans_action(action,self.dim)
    if state==0:
        if next_state == 0:            
            for i in range(self.dim-1):
                tt[i] = -aa[i]
            tt[self.dim-1] = 1-self.delta
    if state==0:
        if next_state == 1:            
            for i in range(self.dim-1):
                tt[i] = aa[i]
            tt[self.dim-1] = self.delta
    if state==1:
        if next_state == 0:            
            for i in range(self.dim-1):
                tt[i] = 0
            tt[self.dim-1] = self.delta
    if state==1:
        if next_state == 1:            
            for i in range(self.dim-1):
                tt[i] = 0
            tt[self.dim-1] = 1-self.delta
    return tt

  def weighted_value_embedding(self, state, action, values):
    return (self.embed(state,action,0)*values[0] + self.embed(state,action,1)*values[1])

  def transition_prob(self, state, action, next_state):
    return np.dot(self.embed(state,action,next_state), self.theta)

  def reward(self, state, action):
    return state

  def step(self, action):
    # make update here
    self.cur_state = np.random.binomial(1, self.transition_prob(self.cur_state, action, 1))
    return self.cur_state

  def reset(self):
    self.cur_state = 0

In [ ]:
# Note that River Swim is a Linear Kernel MDP (specifically a tabular one)
# However, it's a lot easier to create it logically rather than through the Linear Kernel MDP format

class RiverSwim(Env):
  def __init__(self, S, start_reward = 0.005, end_reward = 1, seed = None):
    self.S = S
    self.start_reward = start_reward
    self.end_reward = end_reward

    # The states are numbered 0 through S-1
    self.cur_state = 0

    # The actions are 0 (left) and 1 (right)
    self.num_actions = 2

    self.rng = np.random.default_rng(seed = seed)

  def transition_prob(self, state, action, next_state):
    # Assumes S >= 2

    if state == 0:
      if next_state == 0:
        if action == 0:
          return 1
        else:
          return 0.1
      elif next_state == state + 1:
        if action == 1:
          return 0.9
        else:
          return 0

    elif state == self.S-1:
      if next_state == self.S-1:
        if action == 1:
          return 0.95
        else:
          return 0
      elif next_state == self.S-2:
        if action == 0:
          return 1
        else:
          return 0.05

    else:
      if next_state == state:
        if action == 1:
          return 0.05
        else:
          return 0
      elif next_state == state - 1:
        if action == 0:
          return 1
        else:
          return 0.05
      elif next_state == state + 1:
        if action == 1:
          return 0.9
        else:
          return 0

    return 0


  def reward(self, state, action):
    # The original River Run seems to give a reward for landing on a state, not performing an action
    # I've changed it so that taking an action from the right states gives the reward

    if state == 1:
      return self.start_reward
    elif state == self.S:
      return self.end_reward
    else:
      return 0

  def step(self, action):
    num = self.rng.random()

    if self.cur_state == 0:
      if action == 0:
        pass
      else:
        if num <= 0.9:
          self.cur_state = self.cur_state + 1
    
    elif self.cur_state == self.S-1:
      if action == 0:
        self.cur_state = self.cur_state - 1
      else:
        if num >= 0.95:
          self.cur_state = self.cur_state - 1
    
    else:
      if action == 0:
        self.cur_state = self.cur_state - 1
      else:
        if num <= 0.9:
          self.cur_state = self.cur_state + 1
        elif num >= 0.95:
          self.cur_state = self.cur_state - 1
    
    return self.cur_state

  def reset(self):
    self.cur_state = 0

In [ ]:
######initialization 


def trans_action(a, d):###d is dimension, dimension of a is d-1
    tt = np.zeros(d-1)-1
    bb = bin(a)
    for i in range(len(bb)-2):
        tt[i] = 2*np.double(bb[i+2])-1
    return(tt)

# In this toy model, there are only two states, 0 and 1
# For the parameters of phi, s is the state???, a is the action selected, and sp is the possible next state???
# The returned tt is the embedding in R^d space of this state, action, next state tuple

def phi(s,a,sp):
    tt = np.zeros(dim)
    d = dim
    aa = trans_action(a,d)
    if s==0:
        if sp == 0:            
            for i in range(d-1):
                tt[i] = -aa[i]
            tt[d-1] = 1-delta
    if s==0:
        if sp == 1:            
            for i in range(d-1):
                tt[i] = aa[i]
            tt[d-1] = delta
    if s==1:
        if sp == 0:            
            for i in range(d-1):
                tt[i] = 0
            tt[d-1] = delta
    if s==1:
        if sp == 1:            
            for i in range(d-1):
                tt[i] = 0
            tt[d-1] = 1-delta
    return tt

# phiv is takes in a state action pair and the current values
# it returns a weighted sum of the embeddings of the next states, weighted by the values of those next states
def phiv(s,a,v):
    return (phi(s,a,0)*v[0] + phi(s,a,1)*v[1])

# proba returns the probability that action a will take the agent from state s to state sp
def proba(s,a,sp):
    return np.dot(phi(s,a,sp), thetastar)

# in this toy model, state 0 has reward 0 and state 1 has reward 1
def reward(s,a):
    return s




In [ ]:
def EVI(ite, U, hattheta, beta):########here U should be sigma^{1/2}
    qq = np.ones((STATE, ACTION))/(1-gamma)
    vv = np.ones(STATE)/(1-gamma)
    for i in range(ite):
        for ss in range(STATE):           
            
            for aa in range(ACTION):

                ee = np.zeros(dim-1)
                ee = np.append(ee, 1)
                ee.shape = (1, dim)              
                phi_v = phiv(ss,aa,vv)
                TT = np.diag(np.ones(dim))
                TT = TT[0:dim-1,: ]
                f = -phi_v
                x = cp.Variable(dim)

                soc_constraints = [cp.norm(U@(x-hattheta), 2)<=beta, cp.norm(TT@x, 1)<=delta, ee@x == 1 ]
                
                prob = cp.Problem(cp.Minimize(f.T@x),soc_constraints)
                prob.solve()
                
                qq[ss,aa] = reward(ss,aa)+gamma*phi_v.T@x.value
            vv[ss] = max(qq[ss])
    return qq,vv 

In [ ]:
def EVI_modified(ite, U, hattheta, beta, phiv, reward, STATE, ACTION):########here U should be sigma^{1/2}
    qq = np.ones((STATE, ACTION))/(1-gamma)
    vv = np.ones(STATE)/(1-gamma)
    for i in range(ite):
        for ss in range(STATE):           
            
            for aa in range(ACTION):

                ee = np.zeros(dim-1)
                ee = np.append(ee, 1)
                ee.shape = (1, dim)              
                phi_v = phiv(ss,aa,vv)
                TT = np.diag(np.ones(dim))
                TT = TT[0:dim-1,: ]
                f = -phi_v
                x = cp.Variable(dim)

                soc_constraints = [cp.norm(U@(x-hattheta), 2)<=beta, cp.norm(TT@x, 1)<=delta, ee@x == 1 ]
                
                prob = cp.Problem(cp.Minimize(f.T@x),soc_constraints)
                prob.solve()
                
                qq[ss,aa] = reward(ss,aa)+gamma*phi_v.T@x.value
            vv[ss] = max(qq[ss])
    return qq,vv 

In [ ]:
# A test for EVI
SIGMA = np.random.randn(dim, dim)
hattheta = thetastar
EVI(10, SIGMA,thetastar, 100)

NameError: ignored

In [ ]:
############tabular###################
QVALUE = np.ones((STATE, ACTION))/(1-gamma)
TQVALUE = np.ones((STATE, ACTION))/(1-gamma)
VVALUE = np.ones(STATE)/(1-gamma)
COUNT = np.zeros((STATE, ACTION))
H = 1/(1-gamma)
s_cur = 0
REWARD = 0

for t in range(T):
    mmax = max(QVALUE[s_cur])
    tt = []
    for aa in range(ACTION):
        if QVALUE[s_cur][aa] == mmax:
            tt.append(aa)
    a_cur = tt[randrange(len(tt))]
#     a_cur = randrange(ACTION)
#     a_cur = 127
    s_next = np.random.binomial(1, proba(s_cur, a_cur, 1))
    REWARD += reward(s_cur, a_cur)
    COUNT[s_cur, a_cur] += 1
    kk = COUNT[s_cur, a_cur]
    bonus = 0*np.sqrt(np.log(kk+1)/kk)
    VVALUE[s_next] = max(QVALUE[s_next])
    alpha = (H+1)/(H+kk)
    TQVALUE[s_cur, a_cur] = (1-alpha)*TQVALUE[s_cur, a_cur] + alpha*(reward(s_cur, a_cur) + bonus + gamma*VVALUE[s_next])
    QVALUE[s_cur, a_cur] = min(QVALUE[s_cur, a_cur],TQVALUE[s_cur, a_cur] )
    s_cur = s_next
    print([t, REWARD, a_cur])
#     print(REWARD)

Streaming output truncated to the last 5000 lines.
[5000, 2898, 123]
[5001, 2898, 123]
[5002, 2899, 34]
[5003, 2899, 123]
[5004, 2900, 34]
[5005, 2900, 123]
[5006, 2901, 34]
[5007, 2901, 123]
[5008, 2901, 123]
[5009, 2902, 34]
[5010, 2902, 123]
[5011, 2902, 123]
[5012, 2902, 123]
[5013, 2903, 34]
[5014, 2903, 123]
[5015, 2903, 123]
[5016, 2903, 123]
[5017, 2904, 34]
[5018, 2904, 123]
[5019, 2904, 123]
[5020, 2905, 34]
[5021, 2906, 34]
[5022, 2906, 123]
[5023, 2907, 34]
[5024, 2908, 34]
[5025, 2909, 34]
[5026, 2910, 34]
[5027, 2910, 123]
[5028, 2910, 123]
[5029, 2911, 34]
[5030, 2911, 123]
[5031, 2912, 34]
[5032, 2912, 123]
[5033, 2912, 123]
[5034, 2912, 123]
[5035, 2913, 34]
[5036, 2914, 34]
[5037, 2915, 34]
[5038, 2916, 34]
[5039, 2917, 34]
[5040, 2918, 34]
[5041, 2918, 123]
[5042, 2919, 34]
[5043, 2920, 34]
[5044, 2920, 123]
[5045, 2921, 34]
[5046, 2921, 123]
[5047, 2922, 34]
[5048, 2922, 123]
[5049, 2923, 34]
[5050, 2924, 34]
[5051, 2924, 123]
[5052, 2925, 34]
[5053, 2926, 34]
[5054

In [ ]:
############UCLK######################
QVALUE = np.ones((STATE, ACTION))/(1-gamma)
VVALUE = np.ones(STATE)/(1-gamma)
REWARD = 0
s_cur = 0

SIGMA = np.diag(np.ones(dim))
TSIGMA = np.diag(np.ones(dim))
BB = np.zeros((dim, 1))
t = 1

for k in range(T):
    hattheta = np.linalg.solve(SIGMA, BB)
    hattheta.shape = (dim)
    TSIGMA = copy.deepcopy(SIGMA)
    bonus = 1*np.log(t+1) + np.linalg.norm(thetastar, 2)
    UU = scipy.linalg.sqrtm(SIGMA)
    qq, vv  = EVI(10, UU, hattheta, bonus)
    for ss in range(STATE):
        for aa in range(ACTION):
            QVALUE[ss][aa] = qq[ss][aa]
        VVALUE[ss] = vv[ss]
    while (np.linalg.det(SIGMA)<3*np.linalg.det(TSIGMA)):
        mmax = max(QVALUE[s_cur])
        tt = []
        for aa in range(ACTION):
            if QVALUE[s_cur][aa] == mmax:
                tt.append(aa)
        a_cur = tt[randrange(len(tt))]
#         print(a_cur)
        s_next = np.random.binomial(1, proba(s_cur, a_cur, 1))
        REWARD += reward(s_cur, a_cur)
        print(t)
        print(REWARD)
#         print(np.linalg.det(TSIGMA))
#         print(np.linalg.det(SIGMA))
        phi_v = phiv(s_cur, a_cur, VVALUE)
        phi_v.shape = (dim,1)
        SIGMA +=  np.dot(phi_v, phi_v.T) 
        BB += phi_v*VVALUE[s_next]
        t = t+1
        s_cur = s_next
        if t>T:
            break
    if t>T:
        break


In [ ]:
# variables like dim and gamma would go in the UCLK class, not the environment

# the env parameter is of the class Env, so you should only call transition, transition_prob, and reward
# it's not guaranteed to be Linear Kernel, so you can't call the Linear Kernel functions
class Algorithm(ABC):
  def __init__(self, env):
    self.env = env

  @abstractmethod
  def train(self, timesteps):
    pass

class UCLK(Algorithm):
  def __init__(self, env, d, A, S, gamma, phi):
    super().__init__(env)
    self.d = d
    self.A = A
    self.S = S
    self.gamma = gamma
    self.phi = phi

    self.qvals = np.ones((S, A))/(1-gamma)
    self.vvals = np.ones(S)/(1-gamma)
    self.SIGMA = np.diag(np.ones(dim))
    self.TSIGMA = np.diag(np.ones(dim))
    self.BB = np.zeros((dim, 1))

  def phiv(self, s, a, v):
    total = 0
    for i in range(self.S):
      total += self.phi(s, a, i)*v[i]
    return total

  def train(self, timesteps):
    REWARD = 0
    self.env.reset()
    t = 1

    for k in range(timesteps):
        hattheta = np.linalg.solve(self.SIGMA, self.BB)
        hattheta.shape = (dim)
        self.TSIGMA = copy.deepcopy(self.SIGMA)
        bonus = 1*np.log(t+1) + np.linalg.norm(thetastar, 2)
        UU = scipy.linalg.sqrtm(self.SIGMA)
        qq, vv  = EVI_modified(10, UU, hattheta, bonus, self.phiv, self.env.reward, self.S, self.A)
        for ss in range(STATE):
            for aa in range(self.A):
                self.qvals[ss][aa] = qq[ss][aa]
            self.vvals[ss] = vv[ss]
        while (np.linalg.det(self.SIGMA)<3*np.linalg.det(self.TSIGMA)):
            mmax = max(self.qvals[self.env.cur_state])
            tt = []
            for aa in range(self.A):
                if self.qvals[self.env.cur_state][aa] == mmax:
                    tt.append(aa)
            a_cur = tt[randrange(len(tt))]
    #         print(a_cur)

            REWARD += self.env.reward(self.env.cur_state, a_cur)
            print(t)
            print(REWARD)
    #         print(np.linalg.det(self.TSIGMA))
    #         print(np.linalg.det(self.SIGMA))

            #perhaps phi and phiv functions should be an input to the class?
            phi_v = self.phiv(self.env.cur_state, a_cur, self.vvals)
            phi_v.shape = (dim,1)
            self.SIGMA +=  np.dot(phi_v, phi_v.T) 

            self.env.step(a_cur)

            self.BB += phi_v*self.vvals[self.env.cur_state]
            t = t+1

            if t > timesteps:
                break
        if t > timesteps:
            break
  

In [ ]:
this_instance = DongRuoToy(dim, ACTION, STATE, gamma, thetastar)

def phi_copy(s,a,sp):
    tt = np.zeros(dim)
    d = dim

    ttt = np.zeros(d-1)-1
    bbb = bin(a)
    for i in range(len(bbb)-2):
        ttt[i] = 2*np.double(bbb[i+2])-1
    #aa = trans_action(a,d)
    aa = ttt

    if s==0:
        if sp == 0:            
            for i in range(d-1):
                tt[i] = -aa[i]
            tt[d-1] = 1-delta
    if s==0:
        if sp == 1:            
            for i in range(d-1):
                tt[i] = aa[i]
            tt[d-1] = delta
    if s==1:
        if sp == 0:            
            for i in range(d-1):
                tt[i] = 0
            tt[d-1] = delta
    if s==1:
        if sp == 1:            
            for i in range(d-1):
                tt[i] = 0
            tt[d-1] = 1-delta
    return tt

new_UCLK = UCLK(this_instance, dim, ACTION, STATE, gamma, phi_copy)

In [ ]:
new_UCLK.train(10)

1
0
2
1
3
1
4
2
5
2
6
2
7
2
8
3
9
3
10
4


In [ ]:
def RiverSwimFiveStatePhi(state, action, next_state):
  embedding = np.zeros(5*2*5)
  embedding[ state*2*5 + action*5 + next_state ] = 1
  return embedding

new_river = RiverSwim(5)
swim_UCLK = UCLK(new_river, 5*2*5, 2, 5, gamma, RiverSwimFiveStatePhi)

In [ ]:
swim_UCLK.train(10)

ValueError: ignored